In [20]:
import pandas as pd
import sklearn
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler, PolynomialFeatures, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, ElasticNet


pd.options.display.max_rows = 150
pd.options.display.max_columns = 150

In [21]:
df = pd.read_csv('data_end.csv') # On importe notre dataSet

In [29]:
df.head() # On vérifie si il est bien importé

,prix,chambres,salle_eau,m2_habitable,m2_total,étages,vue_mer,vue,condition,grade,m2_hors_sous_sol,m2_cave,annee_construction,annee_renovation,98001,98002,98003,98004,98005,98006,98007,98008,98010,98011,98014,98019,98022,98023,98024,98027,98028,98029,98030,98031,98032,98033,98034,98038,98039,98040,98042,98045,98052,98053,98055,98056,98058,98059,98065,98070,98072,98074,98075,98077,98092,98102,98103,98105,98106,98107,98108,98109,98112,98115,98116,98117,98118,98119,98122,98125,98126,98133,98136,98144,98146,98148,98155,98166,98168,98177,98178,98188,98198,98199
0,221900.0,3,1.00,109.624675,524.897808,1.0,0,0,3,7,109.624675,0,1955,1955,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,538000.0,3,2.25,238.758826,672.798216,2.0,0,0,3,7,201.597919,400,1951,1991,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,180000.0,2,1.00,71.534745,929.022668,1.0,0,0,3,6,71.534745,0,1933,1933,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,604000.0,4,3.00,182.088443,464.511334,1.0,0,0,5,7,97.547380,910,1965,1965,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,510000.0,3,2.00,156.075808,750.650316,1.0,0,0,3,8,156.075808,0,1987,1987,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
# On sépare notre dataSet
X = df.iloc[0:,1:]
y = df['prix']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [24]:
# On créé notre pipeline
model = make_pipeline(RobustScaler(), PolynomialFeatures(degree=2),Lasso(alpha=32.142857142857146)) 

# On entraine notre model
model.fit(X_train, y_train)

# On calcul notre score
model.score(X_test, y_test)

/Users/thomasdulompont/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.160e+13, tolerance: 1.865e+11
  model = cd_fast.enet_coordinate_descent(


0.8910488680203735

In [25]:
# On retire nos plus grosses erreurs
residus = model.predict(X_train)
residus = abs(residus - y_train)

test = residus.loc[residus > residus.quantile(0.96)].index

X_train = X_train.drop(test)
y_train = y_train.drop(test)

In [26]:
# On recommence nos test sans les residus
model.fit(X_train, y_train)

# On calcul notre score sans les residus
model.score(X_test, y_test)

/Users/thomasdulompont/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.171e+13, tolerance: 1.164e+11
  model = cd_fast.enet_coordinate_descent(


0.8955202678994019

In [27]:
# On recherche notre meilleur alpha (paramètre de pipeline)

# from sklearn.model_selection import RandomizedSearchCV
# import numpy as np

# model = make_pipeline(RobustScaler(), PolynomialFeatures(degree=2),Lasso())
# parameters = {
#     "lasso__alpha":np.linspace(0, 40)
# }
# randm_src = RandomizedSearchCV(estimator=model, param_distributions = parameters, n_iter = 5)
# randm_src.fit(X_train, y_train)

# print(randm_src.best_params_)

In [28]:
# On exporte notre IA pour qu'elle soit plus rapide à utiliser
import pickle

pickle_out = open("modelRid.pkl", "wb")
pickle.dump(model, pickle_out)
pickle_out.close()